In [1]:
# !pip install pandas
# !pip install torch
# !pip install tqdm  # pip install ipywidgets or something
!pip install wandb &> /dev/null
# !pip install gensim &> /dev/null
!pip install torchmetrics &> /dev/null

In [2]:
USING_WANDB = True  # Set to false if not Peter

if USING_WANDB:
    import wandb
    wandb.login();
    # !wandb login

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_folder = "/content/drive/MyDrive/dna_ml_model_data"
accessible_file = f"{data_folder}/accessible.fasta"
notaccessible_file = f"{data_folder}/notaccessible.fasta"
reduced_nonaccessible_file = f"{data_folder}/reduced_nonaccessible.fasta"
comp_file = f"{data_folder}/test.fasta"
rest_notaccessible_file = f"{data_folder}/rest_nonaccessible.fasta"

DATA_ZIP_FILE = "Files.zip"
LINES_PER_SEQUENCE = 4
ACCESSIBLE_LABEL = 1
NOT_ACCESSIBLE_LABEL = 0
TEMP_SPLIT = 0.30  # validation + test
TEST_SPLIT = 0.15
PRETRAINED_DIR = "pretrained"
SOLUTION_FILE = "predictions.csv"

In [87]:
import dna_dataset, utils, CNNModel # , LSTMCNNModel, constants
import torch.nn as nn, torch.optim as optim, torch
from tqdm.notebook import tqdm
import importlib, os, numpy as np, pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from torchmetrics.functional import f1_score
from sklearn.metrics import confusion_matrix, precision_recall_curve

importlib.reload(dna_dataset)
importlib.reload(utils)
importlib.reload(CNNModel)
# importlib.reload(LSTMCNNModel)
# importlib.reload(constants)

<module 'CNNModel' from '/content/CNNModel.py'>

In [ ]:
# Unzip the zip file
!unzip $DATA_ZIP_FILE

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
# Read the data from data files
acc_sequences, acc_labels, _ = dna_dataset.read_data_file(accessible_file, accessible=True, shuffle=True)
# not_sequences, not_labels, _ = dna_dataset.read_data_file(reduced_nonaccessible_file, accessible=False, shuffle=True)

In [8]:
not_sequences, not_labels, _ = dna_dataset.read_data_file(notaccessible_file, accessible=False, shuffle=True)

In [9]:
num_acc = len(acc_sequences)
num_not = len(not_sequences)
full_size = num_acc + num_not

# Split accessible data into train val test split
acc_seq_train, acc_seq_temp, acc_lab_train, acc_lab_temp = train_test_split(acc_sequences, acc_labels, test_size=TEMP_SPLIT, random_state=1)
acc_seq_val, acc_seq_test, acc_lab_val, acc_lab_test = train_test_split(acc_seq_temp, acc_lab_temp, test_size=TEST_SPLIT/TEMP_SPLIT, random_state=1)

# Split not accessible data into train val test split
not_seq_train, not_seq_temp, not_lab_train, not_lab_temp = train_test_split(not_sequences, not_labels, test_size=TEMP_SPLIT, random_state=1)
not_seq_val, not_seq_test, not_lab_val, not_lab_test = train_test_split(not_seq_temp, not_lab_temp, test_size=TEST_SPLIT/TEMP_SPLIT, random_state=1)

# assert(self.accessible_count + self.not_accessible_count == len(self.sequences))
assert(len(acc_sequences) == len(acc_labels) == len(acc_seq_train) + len(acc_seq_val) + len(acc_seq_test))
assert(len(not_sequences) == len(not_labels) == len(not_lab_train) + len(not_lab_val) + len(not_lab_test))

Upsample or Downsample the dataset  
(AFTER splitting to train/validation/test to prevent data leakage)

In [ ]:
UPSAMPLE = False
DOWNSAMPLE = True


if UPSAMPLE:
    num_diff = len(not_seq_train) - len(acc_seq_train)
    # upscale the minority
    upsampled_acc_seq, upsampled_acc_lab = resample(acc_seq_train, acc_lab_train, n_samples=num_diff, random_state=1)
    acc_seq_train += upsampled_acc_seq
    acc_lab_train += upsampled_acc_lab

elif DOWNSAMPLE:
    # downsample the majority
    downsampled_not_seq, downsampled_not_lab = resample(not_seq_train, not_lab_train, n_samples=len(acc_seq_train), random_state=1)
    not_seq_train += downsampled_not_seq
    not_lab_train += downsampled_not_lab

    # consider upweighting for calibration

    # consider adding rest of data to test and val

print(len(acc_seq_train), len(acc_lab_train))
assert(len(acc_seq_train) == len(acc_lab_train))  # ensure the data is now equal

In [10]:
# Create the (shuffled) datasets, after appending the lists
sequences_train = acc_seq_train + not_seq_train
labels_train = acc_lab_train + not_lab_train
# sequences_train, labels_train = dna_dataset.shuffle_lists(sequences_train, labels_train)
# df_train = pd.DataFrame({"sequences": sequences_train, "labels": labels_train})
train_dataset = dna_dataset.DNADataset(sequences_train, labels_train)
num_train = len(train_dataset)
assert(len(sequences_train) == len(labels_train) == num_train)

sequences_val = acc_seq_val + not_seq_val
labels_val = acc_lab_val + not_lab_val
# sequences_val, labels_val = dna_dataset.shuffle_lists(sequences_val, labels_val)
# df_val = pd.DataFrame({"sequences": sequences_val, "labels": labels_val})
val_dataset = dna_dataset.DNADataset(sequences_val, labels_val)

sequences_test = acc_seq_test + not_seq_test
labels_test = acc_lab_test + not_lab_test
# sequences_test, labels_test = dna_dataset.shuffle_lists(sequences_test, labels_test)
# df_test = pd.DataFrame({"sequences": sequences_test, "labels": labels_test})
test_dataset = dna_dataset.DNADataset(sequences_test, labels_test)

# assert(len(sequences_train) + len(sequences_val) + len(sequences_test) == full_size)
# full_dataset = dna_dataset.DNADataset(accessible_file, reduced_nonaccessible_file)

In [11]:
# ensure the DNADataset is loaded properly
print('total sequences', full_size)
print('num accessible', num_acc)
print('num not accessible', num_not)
i = 0
print(f"example entry {i}")
item = train_dataset[i]
# print(item['sequence'])  # long answer
print("label", item['label'])
# ensure dataset was shuffled properly
# check that not all the accessible labels are at the front
for i in range(len(train_dataset)):
    if train_dataset[i]['label'] != ACCESSIBLE_LABEL:
        print('shuffled')
        break

# Even
# total sequences 94478
# num accessible 47239
# num not accessible 47239

# Full (imbalanced)
# total sequences 525688
# num accessible 47239
# num not accessible 478449

total sequences 525688
num accessible 47239
num not accessible 478449
example entry 0
label 0
shuffled


In [ ]:
# # Free up memory
# del sequences_train, labels_train, sequences_val, labels_val, sequences_test, labels_test
# del acc_seq_train, not_seq_train, acc_lab_train, not_lab_train
# del acc_seq_val, not_seq_val, acc_lab_val, not_lab_val
# del acc_seq_test, not_seq_test, acc_lab_test, not_lab_test

In [88]:
# Initialize model
importlib.reload(CNNModel)
torch.manual_seed(0)

kernel_size = 3     # 2  # should use odd size
embed_dim = 4  # 4
num_filters1 = 64  # 128
num_filters2 = 128  # 64
pool_kernel_size = 2  # 2
hidden_dense1 = 64  # 128
hidden_dense2 = 32  # 64
dropout_rate_Dense = 0.5  # .5

# insert torch model here, that takes sequence as input and output a label 0 or 1
model = CNNModel.CNNModel(kernel_size,
                           embed_dim,
                           num_filters1,
                           num_filters2,
                           pool_kernel_size,
                           hidden_dense1,
                           hidden_dense2,
                           dropout_rate_Dense
                           )

# importlib.reload(LSTMCNNModel)
# model = LSTMCNNModel.LSTMCNNModel(
#                             kernel_size=2,
#                             embed_dim=4,
#                             num_filters1=128,
#                             num_filters2=64,
#                             pool_kernel_size=2,
#                             hidden_dense1=128,
#                             hidden_dense2=64,
#                             dropout_rate_Dense=0.5,
#                             lstm_units=1
# )

model.to(device);  # quiet output

In [89]:
# weights for loss function
USE_WEIGHTED_LOSS = False  # Note may not work correctly if used with resampling
# USE_WEIGHTED_LOSS = False

# weight_for_class_i = total_samples / (num_samples_in_class_i * num_classes)
weight_class0 = num_train / len(acc_seq_train) * 2
weight_class1 = num_train / len(not_seq_train) * 2
print("weights for class 0", weight_class0)
print("weights for class 1", weight_class1)

weights for class 0 22.256690960776606
weights for class 1 2.1974656180392578


In [90]:
# parameters
epochs = 20
batch_size = 64
learning_rate = .0001
if USE_WEIGHTED_LOSS:
    loss_fn = nn.BCELoss()
    # loss_fn = nn.BCELoss(weight=torch.Tensor([weight_class0, weight_class1]))
else:
    loss_fn = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)  # weight_decay=1
# optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=.01)  # weight_decay=1
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)  # pretty bad

In [91]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, # shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, # shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, # shuffle=True
)

In [92]:
# del train_dataset, val_dataset, test_dataset

In [93]:
# USING_WANDB = True

In [94]:
# TRAINING LOOP
USING_WANDB = False
if USING_WANDB:
    note = f"weighted_loss_unbalanced_data"
    # note = f"{optimizer.__class__.__name__}-{kernel_size}-kernel-{num_filters1}-{num_filters2}-conv-{hidden_dense1}-{hidden_dense2}-dense" # "lstm_no_dropout" # "0.5_dropout"
    wandb.init(
        project="dna_ml_model",
        # name=f"experiment_{run}"
        name=f"{note}",
        settings=wandb.Settings(start_method="fork"),
        config = {
            "architecture": model.__class__.__name__,
            "epochs": epochs,
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "optimizer": optimizer.__class__.__name__,
            "notes": note
    })


# step = 0
best_acc = 0.0

for epoch in range(epochs):
    print(f"Epoch {epoch + 1} of {epochs}")

    running_loss = 0.0
    running_corrects = 0.0
    # total_outputs = torch.empty(0).to(device)
    # total_labels = torch.empty(0).to(device)

    CM = 0  # confusion matrix

    model.train()
    for batch in train_loader:  # tqdm()
        # Forward propagate
        samples, labels = batch["sequence"].to(device), batch['label'].to(device)

        outputs = model(samples).flatten()

        labels = labels.reshape(-1,1).float().flatten()

        # Backpropagation and gradient descent

        if USE_WEIGHTED_LOSS:
            acc_weight = (labels==ACCESSIBLE_LABEL) * weight_class0
            not_weight = (labels==NOT_ACCESSIBLE_LABEL) * weight_class1
            acc_weight[acc_weight==0] = 1
            not_weight[not_weight==0] = 1

            outputs *= acc_weight
            outputs *= not_weight

        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()  # reset gradients before next iteration

        running_loss += loss.item() * samples.size(0)  # loss per sample times batch size

        preds = torch.round(outputs)
        running_corrects += torch.sum(preds == labels.data).item()
        # print(outputs.shape, preds.shape)
        CM += confusion_matrix(labels.flatten().detach().numpy(), preds.flatten().detach().numpy())

    #     total_outputs = torch.cat((total_outputs, outputs))
    #     total_labels = torch.cat((total_labels, labels))
    # f1 = f1_score(total_outputs.flatten(), total_labels.flatten(), task="binary", num_classes=2).item()

    # Calculate recall, precision, f1 score
    acc_score, precision, recall, f1 = utils.compute_metrics(CM)

    # if step % n_eval == 0:
    # Compute training loss and accuracy.
    with torch.no_grad():
        model.eval()
        # Compute validation loss and accuracy.
        # accuracy = utils.compute_accuracy(outputs, labels)  # only does current batch
        val_loss, val_acc, val_precision, val_recall, val_f1 = utils.evaluate(val_loader, model, loss_fn, device)

    epoch_loss = running_loss / num_train
    epoch_acc = running_corrects / num_train
    print(f"""Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1: {f1:.4f}
          Precision {precision:.4f} Recall: {recall:.4f}""")
    print(f"""Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f} F1: {val_f1:.4f}
          Precision {val_precision:.4f} Recall: {val_recall:.4f}""")

    # # deep copy the model
    # if val_accuracy > best_acc:
    #     best_acc = epoch_acc
    #     torch.save(model.state_dict(), best_model_params_path)

    if USING_WANDB:
        wandb.log({"Train Loss": epoch_loss,
                    "Train Acc": epoch_acc,
                    "Val Loss": val_loss,
                    "Val Acc": val_acc,
                    "Train Precision": precision,
                    "Val Precision": precision,
                    "Train F1": f1,
                    "Val F1": val_f1
                  #  "Epoch": epoch
        })
        # step += 1

    print()

Epoch 1 of 20
acc_score 0.9120374470238471 accuracy 0.9134465397638644
f1 0.17432608953805961 f1_2 0.16411513090133667
Training Loss: 0.4823 Acc: 0.9105 F1: 0.0765
              Precision 0.5276 Recall: 0.0413
Validation Loss: 398.8454 Acc: 0.9134 F1: 0.1743
              Precision 0.6081 Recall: 0.1017

Epoch 2 of 20
acc_score 0.9134290593965463 accuracy 0.9148415405881831
f1 0.20494945974207038 f1_2 0.1959046870470047
Training Loss: 0.4708 Acc: 0.9120 F1: 0.1200
              Precision 0.5910 Recall: 0.0667
Validation Loss: 361.5194 Acc: 0.9148 F1: 0.2049
              Precision 0.6332 Recall: 0.1223

Epoch 3 of 20
acc_score 0.9139730533240559 accuracy 0.9153868590922349
f1 0.19621749408983455 f1_2 0.18673817813396454
Training Loss: 0.4660 Acc: 0.9127 F1: 0.1461
              Precision 0.5977 Recall: 0.0832
Validation Loss: 338.8538 Acc: 0.9154 F1: 0.1962
              Precision 0.6661 Recall: 0.1151

Epoch 4 of 20
acc_score 0.9151242962869252 accuracy 0.9165409052287167
f1 0.2582642

/content/utils.py:42: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2 * precision * recall / (precision + recall)


acc_score 0.9149851350496553 accuracy 0.9164014051462849
f1 0.20133111480865226 f1_2 0.19195881485939026
Training Loss: 0.4573 Acc: 0.9139 F1: 0.1787
              Precision 0.6228 Recall: 0.1043
Validation Loss: 318.4098 Acc: 0.9164 F1: 0.2013
              Precision 0.7058 Recall: 0.1174

Epoch 6 of 20
Training Loss: 0.4565 Acc: 0.9140 F1: 0.1902
              Precision 0.6150 Recall: 0.1125
Validation Loss: 342.9195 Acc: 0.9178 F1: 0.3476
              Precision 0.6069 Recall: 0.2436

Epoch 7 of 20
Training Loss: 0.4537 Acc: 0.9146 F1: 0.2020
              Precision 0.6247 Recall: 0.1205
Validation Loss: 331.6018 Acc: 0.9193 F1: 0.3175
              Precision 0.6616 Recall: 0.2089

Epoch 8 of 20


KeyboardInterrupt: 

In [23]:
# wandb.finish()

In [ ]:
# Precision recall plot for finding optimal threshold

which_dataset = "val"  # test
# which_dataset = "test"
dataset = val_dataset if which_dataset == "val" else test_dataset
dataloader = val_loader if which_dataset == "val" else test_loader

total_probs = np.empty(0)
model.eval()
# Calculate the probs
for batch in dataloader:
    samples, labels = batch['sequence'].to(device), batch['label'].to(device)

    outputs = model(samples)

    total_probs = np.concatenate((total_probs, outputs.flatten().detach().numpy()))


precisions, recalls, thresholds = precision_recall_curve(dataset.list2, total_probs)
# dataset.list2  # list of all labels

import matplotlib.pyplot as plt
plt.plot(recalls, precisions, marker='.', label=which_dataset, color="black")
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend()
fscore = (2 * precisions * recalls) / (precisions + recalls)

# Turn nan values to 0
fscore[np.isnan(fscore)] = 0

# locate the index of the largest f score
ix = np.argmax(fscore)
print(ix)
best_threshold = thresholds[ix]
plt.plot(recall[ix], precision[ix], color="red")
plt.axvline(recall[ix], ls='--', color="red", label="Best F-Score")
# show the plot
plt.show()
print("Best Threshold", thresholds[ix], " with F-Score", fscore[ix])

In [ ]:
# EVALUATE on test dataset
total_predictions = 0
total_correct = 0

model.eval()

CM = 0  # confusion matrix

for batch in test_loader:  # tqdm()

    test_samples, test_labels = batch['sequence'].to(device), batch['label'].to(device)
    test_outputs = model(test_samples)
    test_preds = torch.round(test_outputs)

    test_labels = test_labels.reshape(-1, 1).float()

    test_loss = loss_fn(test_outputs, test_labels).item()  # change tensor to single val

    total_correct += (test_preds == test_labels).sum().item()

    total_predictions += len(test_outputs)

    CM += confusion_matrix(test_labels.flatten().detach().numpy(), test_preds.flatten().detach().numpy())

# Computer accuracy, precision, recall, and f1 metrics
tn, tp, fp, fn = CM[0][0], CM[1][1], CM[0][1], CM[1][0]  # true negative, ... false positive, etc
acc_score = (tp + tn)/ (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)


# model.train()
test_accuracy = total_correct / total_predictions
assert(test_accuracy == acc_score)
# test_f1_score = f1_score(total_outputs.flatten(), total_labels.flatten(), task="binary", num_classes=2).item()
print(f"Final Test Accuracy: {test_accuracy * 100} %")
print(f"Final Test Precision: {precision * 100} %")
print(f"Final Test Recall: {recall * 100} %")
print(f"Final Test F1 Score: {f1 * 100} %")

if USING_WANDB:
    wandb.summary['test_accuracy'] = test_accuracy
    wandb.summary['test_precision'] = precision
    wandb.summary['test_recall'] = recall
    wandb.summary['test_f1_score'] = f1

    # wandb.finish()

Final Test Accuracy: 76.89105278012983 %
Final Test Precision: 74.07454200884396 %
Final Test Recall: 82.74061529777025 %
Final Test F1 Score: 78.16812212519164 %


In [ ]:
# from sklearn.metrics import roc_curve, plot_roc_curve,


In [ ]:
# Potential solutions to out of gpu memory
# import gc
# gc.collect()

# with torch.no_grad():
#     torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)

# del total_outputs, total_labels

In [ ]:
# test on rest of nonaccessible
rest_not_sequences, rest_not_labels, _ = dna_dataset.read_data_file(rest_notaccessible_file, accessible=False, shuffle=True)

In [ ]:
rest_nonacc_dataset = dna_dataset.DNADataset(rest_not_sequences, rest_not_labels)
rest_nonacc_loader = torch.utils.data.DataLoader(rest_nonacc_dataset, batch_size=batch_size)#, shuffle=True)

In [ ]:
# test on rest of nonaccessible
probs_out = []  # tuples of probability, id

model.eval()

for batch in rest_nonacc_loader:  # tqdm()
    samples, _ = batch["sequence"].to(device), batch['label']  # not a tensor
    outputs = model(samples)
    out_list = outputs.tolist()

    for i in range(len(out_list)):
        probs_out.append(out_list[i])

probs_out = np.array(probs_out)
falses = len(probs_out[probs_out<0.5])
print("correctly predicted ", falses, " non_accessible values out of ", len(probs_out), " total")
rest_non_correct = falses / len(probs_out)

print(rest_non_correct, " correct (all not accessible)")

wandb.summary['rest_nonacc_accuracy'] = rest_non_correct

wandb.finish()

Save Model

In [ ]:
# Create pretrained directory if not yet created
if not os.path.isdir(PRETRAINED_DIR):
    os.mkdir(PRETRAINED_DIR)

now = datetime.now()
datetime_str = now.strftime("%m-%d-%H-%M-%S")
model_save_path = os.path.join(
    PRETRAINED_DIR,
    f'{note}{datetime_str}-{model.__class__.__name__}-model-{learning_rate}lr-{epochs}epochs.pt'
)
print('model_save_path', model_save_path)
CNNModel.save_CNNModel(model_save_path, model)
print(f"model saved at {datetime_str}")

model_save_path pretrained/proper_rest_notacc01-04-20-24-24-CNNModel-model-0.0001lr-20epochs.pt
model saved at 01-04-20-24-24


Load Model

In [ ]:
model_save_path = "/content/pretrained/proper_rest_notacc01-04-20-24-24-CNNModel-model-0.0001lr-20epochs.pt"
model = CNNModel.load_CNNModel(model_save_path)
model.to(device);

Inference on Competition File

In [ ]:
# Load Competitation Data
# Set shuffle=False to save time, unnecessary for comp file
comp_sequences, _, comp_ids = dna_dataset.read_data_file(comp_file, labeled=False, shuffle=False)
competition_dataset = dna_dataset.DNADataset(comp_sequences, comp_ids, comp=True)
competition_loader = torch.utils.data.DataLoader(
    competition_dataset, batch_size=batch_size, # shuffle=True
)
print("Competition dataset loaded.")

Competition dataset loaded.


In [ ]:
probs = []  # tuples of probability, id

model.eval()
for batch in competition_loader:  # tqdm()

    samples, ids = batch["sequence"].to(device), batch['id']  # not a tensor

    outputs = model(samples)

    out_list = outputs.tolist()

    for i in range(len(out_list)):
        probs.append((out_list[i], ids[i]))

print("Finished inference")

Finished inference


In [ ]:
# View the results
np_probs = np.array(list(zip(*probs))[0])
print("Predicted", len(np_probs[np_probs>0.5]), "true values out of ", len(np_probs), " total")
not_zero = np_probs[np_probs<=0.5]
not_zero.sort()
not_one = np_probs[np_probs>0.5]
not_one[::-1].sort()
print("not accessible probs", not_zero)
print("accessible probs", not_one)

# print(np_probs[np_probs>0.0 and np_probs<1.0])

Predicted 86495 true values out of  269315  total
not accessible probs [7.72003546e-07 1.22001074e-06 1.32544631e-06 ... 4.99994636e-01
 4.99995589e-01 4.99998689e-01]
accessible probs [0.99668294 0.99629837 0.99512321 ... 0.500009   0.50000852 0.50000155]


In [ ]:
probs.sort(reverse=True)
print("first 10\n", list(zip(*probs[:10]))[0])
print("last 10 of top 10000\n", list(zip(*probs[9990:10000]))[0])  # probs only

highest_probs = probs[:10000]  # top 10,000

with open(SOLUTION_FILE, "w") as f:
    for pair in highest_probs:
        f.write(pair[1])
        # f.write("a")
        f.write("\n")

first 10
 ([0.9966829419136047], [0.9962983727455139], [0.9951232075691223], [0.99391770362854], [0.9935550093650818], [0.9931140542030334], [0.9928942322731018], [0.9927329421043396], [0.9923359751701355], [0.9920458793640137])
last 10 of top 10000
 ([0.8520146012306213], [0.852008044719696], [0.8520078063011169], [0.8519814610481262], [0.8519793152809143], [0.8519764542579651], [0.8519759178161621], [0.8519702553749084], [0.8519678115844727], [0.8519659042358398])


In [ ]:
zip_file_name = "predictions.zip"
!zip $zip_file_name $SOLUTION_FILE

  adding: predictions.csv (deflated 65%)


In [ ]:
# ONLY for use on google colab. download files
from google.colab import files
import os
dir = 'pretrained'
model_file = os.path.join(dir, os.listdir(dir)[0])
files.download(model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>